In [ ]:
# !git clone https://github.com/pettepiero/Dial-a-ride.git
# %cd Dial-a-ride/
# !pip install virtualenv
# !virtualenv vrpenv # To set up the env
# !source /content/Dial-a-ride/vrpenv/bin/activate
# !source /content/Dial-a-ride/vrpenv/bin/activate

In [ ]:
import copy
import random
from types import SimpleNamespace
from typing import List
import pandas as pd
import vrplib

from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd

from alns import ALNS
from alns.accept import RecordToRecordTravel
from alns.select import *
from alns.stop import MaxIterations


from cvrptw.myvrplib.myvrplib import plot_solution, plot_data, solution_times_statistics, LOGGING_LEVEL
# from cvrptw.myvrplib import plot_solution, plot_data, solution_times_statistics, LOGGING_LEVEL
from cvrptw.myvrplib.data_module import END_OF_DAY, read_solution_format
# from cvrptw.myvrplib.data_module import d_data as d_data
from cvrptw.myvrplib.data_module import data as full_data
from cvrptw.myvrplib.route import Route
from cvrptw.myvrplib.vrpstates import CvrptwState
from cvrptw.initial_solutions.initial_solutions import nearest_neighbor_tw, time_neighbours
from cvrptw.operators.destroy import *
from cvrptw.operators.repair import *
from cvrptw.operators.wang_operators import *
from cvrptw.output.analyze_solution import verify_time_windows
from cvrptw.myvrplib.data_module import read_cordeau_data

In [ ]:
%matplotlib inline
plt.style.use('seaborn-v0_8-colorblind')
title_dict = {"fontsize": 25, "fontweight": "bold"}
labels_dict = {"fontsize": 12, "fontweight": "bold"}
legend_dict = {"fontsize": 15}
SEED = 1234
NUM_ITERATIONS = 200

In [ ]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=LOGGING_LEVEL)

# Select data path
Recommended: (without spaces)

`path_to_cloned_folder + "/data/"`

In [ ]:
data_path = "/home/pettepiero/test2/Dial-a-ride-main/data/"
data = read_cordeau_data(data_path + "c-mdvrptw/pr12")

# Implementation of wang 2024
https://www.sciencedirect.com/science/article/pii/S0360835224002432?via%3Dihub

NOTE: data['dimension'] is the number of customers only, not including depots

In [ ]:
plot_data(data, idx_annotations=True)

In [ ]:
plot_data(full_data, idx_annotations=True)

In [ ]:
print(data)

## Solution state

In [ ]:
def get_customer_info(data, state: CvrptwState, idx: int, cordeau: bool = False):
    """
    Get the customer information for the passed-in index.
    """
    if cordeau:
        if idx == 0:
            print("Error: in Cordeau notation index 0 is a fake customer")
            return
        
    route = state.find_route(idx)
    if route is not None:
        index_in_route = state.find_index_in_route(idx, route)
        route_index = state.routes.index(route)
        print(f"index_in_route: {index_in_route}")
        print(f"route: {route}")

        dict = {
            "index": idx,
            "coords": data["node_coord"][idx],
            "demand": data["demand"][idx].item(),
            "ready time": data["time_window"][idx][0].item(),
            "due time": data["time_window"][idx][1].item(),
            "service_time": data["service_time"][idx].item(),
            "route": route,
        }
    else:
        print(f"Customer {idx} is not in any route")
        dict = {
            "index": idx,
            "coords": data["node_coord"][idx],
            "demand": data["demand"][idx].item(),
            "ready time": data["time_window"][idx][0].item(),
            "due time": data["time_window"][idx][1].item(),
            "service_time": data["service_time"][idx].item(),
            "route": None,
        }
    return dict

## Destroy operators

In [ ]:
degree_of_destruction = 0.05
customers_to_remove = int((data["dimension"] - 1) * degree_of_destruction)
print(f"Removing {customers_to_remove} customers.")

## Repair operators


## Initial solution
We need an initial solution that is going to be destroyed and repaired by the ALNS heuristic. To this end, we use a simple *nearest neighbor (NN)* heuristic. NN starts with an empty solution and iteratively adds the nearest customer to the routes. If there are no routes available, then a new route is created.

### Choosing starting depot
If the number of vehicles if larger than number of depots we split the number of vehicles between the depots.

Otherwise, we choose randomly a depot and generate a route from it.
NOTE: maybe performance of the model can be improved by changing the above policy

In [ ]:
# calculate_depots(data)
print(data['depot_to_vehicles'])
print(data['vehicle_to_depot'])
print(data['dimension'])

In [ ]:
print(data['depots'])

In [ ]:
initial_solution = nearest_neighbor_tw(initial_time_slot=False)

plot_solution(data, initial_solution, "Nearest neighbor solution")

In [ ]:
for route in initial_solution.routes:
    print(route.customers_list)

In [ ]:
initial_solution_stats = solution_times_statistics(data, initial_solution)
print(initial_solution_stats)

## Heuristic solution

Let's now construct our ALNS heuristic. Since we only have one destroy and repair operator, we do not actually use any adaptive operator selection -- but you can easily add more destroy and repair r_operators. 

In [ ]:
# alns = ALNS(rnd.default_rng(SEED))
alns = ALNS(rnd.default_rng())

alns.add_destroy_operator(random_removal)
alns.add_destroy_operator(random_route_removal)
alns.add_destroy_operator(cost_reducing_removal)
alns.add_destroy_operator(worst_removal)

alns.add_destroy_operator(exchange_reducing_removal)
# alns.add_destroy_operator(shaw_removal)   #to be implemented

alns.add_repair_operator(greedy_repair_tw)
alns.add_repair_operator(wang_greedy_repair)

In [ ]:
num_iterations = NUM_ITERATIONS
init = nearest_neighbor_tw(initial_time_slot=False)
select = RouletteWheel([25, 5, 1, 0], 0.8, 5, 2)
accept = RecordToRecordTravel.autofit(
    init.objective(), 0.02, 0, num_iterations
)
stop = MaxIterations(num_iterations)
result, destruction_counts, insertion_counts, d_operators_log, r_operators_log = alns.iterate(init, select, accept, stop, data=data, save_plots=False)

In [ ]:
d_operators = tuple([op[0] for op in alns.destroy_operators])
print(d_operators)
d_ops_dict = {i: op for i, op in enumerate(d_operators)}

In [ ]:
cumulative_sums = np.cumsum(destruction_counts, axis=0)  # Plot each column
rows = np.arange(destruction_counts.shape[0])
fig, ax = plt.subplots(figsize=(10,10))
for col_idx in range(destruction_counts.shape[1]-1):
    plt.plot(rows, cumulative_sums[:, col_idx], label=f"{d_operators[col_idx]}")

# Customize plot
plt.xlabel("Iteration number", fontdict=labels_dict)
plt.ylabel("Number of removals", fontdict=labels_dict)
plt.title("Number of removals by destroy operator", fontdict=title_dict)
plt.legend(**legend_dict)
plt.grid(True)
plt.show()

In [ ]:
r_operators = tuple([op[0] for op in alns.repair_operators])
r_ops_dict = {i: op for i, op in enumerate(r_operators)}
print(r_operators)
cumulative_sums = np.cumsum(insertion_counts, axis=0)  # Plot each column
rows = np.arange(insertion_counts.shape[0])
fig, ax = plt.subplots(figsize=(10, 10))
for col_idx in range(insertion_counts.shape[1] - 1):
    plt.plot(rows, cumulative_sums[:, col_idx], label=f"{r_operators[col_idx]}")

# Customize plot
plt.xlabel("Iteration number", fontdict=labels_dict)
plt.ylabel("Number of insertions", fontdict=labels_dict)
plt.title("Number of insertions by insertion operator", fontdict=title_dict)
plt.legend(**legend_dict)
plt.grid(True)
plt.show()

## Plotting the destroy and repair operators applications

In [ ]:
destroy_operators_log_array = np.zeros(shape=(len(d_operators_log), len(d_operators)), dtype=int)
for i, op in enumerate(d_operators_log):
    destroy_operators_log_array[i, op] +=1
destroy_operators_log_array = np.cumsum(destroy_operators_log_array, axis=0)

In [ ]:
rows = np.arange(destroy_operators_log_array.shape[0])
fig, ax = plt.subplots(figsize=(10, 10))
for col_idx in range(destroy_operators_log_array.shape[1]):
    plt.plot(
        rows, destroy_operators_log_array[:, col_idx], label=f"{d_operators[col_idx]}"
    )

# Customize plot
plt.xlabel("Iteration number", fontdict=labels_dict)
plt.ylabel("Number of applications", fontdict=labels_dict)
plt.title("Number of destroy operator applications", fontdict=title_dict)
plt.legend(**legend_dict)
plt.grid(True)
plt.show()

In [ ]:
results_df = pd.DataFrame(result.statistics.destroy_operator_counts)
reasons = ["Global best", "Better", "Accepted" , "Rejected"]
x = np.arange(len(reasons))
width = 0.20
multiplier = 0

fig, ax = plt.subplots(layout='constrained', figsize=(6, 6))
fig.tight_layout()
for attribute, measurement in results_df.items():
    offset = width*multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_ylabel('Count')
ax.set_title('Destroy operator counts')
ax.set_xticks(x + width, reasons)
ax.legend(loc='right', ncols=1)
# ax.set_xlim(0, 4)

plt.show()

In [ ]:
repair_operators_log_array = np.zeros(
    shape=(len(r_operators_log), len(r_operators)), dtype=int
)
for i, op in enumerate(r_operators_log):
    repair_operators_log_array[i, op] += 1
repair_operators_log_array = np.cumsum(repair_operators_log_array, axis=0)

In [ ]:
rows = np.arange(repair_operators_log_array.shape[0])
fig, ax = plt.subplots(figsize=(10, 10))
for col_idx in range(repair_operators_log_array.shape[1]):
    plt.plot(
        rows, repair_operators_log_array[:, col_idx], label=f"{d_operators[col_idx]}"
    )

# Customize plot
plt.xlabel("Iteration number", fontdict=labels_dict)
plt.ylabel("Number of applications", fontdict=labels_dict)
plt.title("Number of insertion operator applications", fontdict=title_dict)
plt.legend(**legend_dict)
plt.grid(True)
plt.show()

In [ ]:
results_df = pd.DataFrame(result.statistics.repair_operator_counts)
print(results_df)

reasons = ["Global best", "Better", "Accepted", "Rejected"]
x = np.arange(len(reasons))
width = 0.25
multiplier = 0

fig, ax = plt.subplots(layout="constrained")

for attribute, measurement in results_df.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_ylabel("Count")
ax.set_title("Repair operator counts")
ax.set_xticks(x + width, reasons)
ax.legend(loc="upper left", ncols=4)
ax.set_xlim(0, 4)

## Overall results

In [ ]:
solution = result.best_state
objective = solution.objective()
print(f"Best heuristic objective is {objective}.")

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))
result.plot_objectives(ax=ax)

In [ ]:
plot_solution(data, initial_solution, "Nearest-neighbor-solution", save=True, figsize=(8, 8))
plot_solution(data, solution, "Heuristic-solution", idx_annotations=False, save=True, figsize=(8, 8))

In [ ]:
plot_solution(data, initial_solution, "Nearest-neighbor-solution", save=False, figsize=(8, 10))
plot_solution(data, solution, "Heuristic-solution", idx_annotations=False, save=False, figsize=(8, 10))

In [ ]:
print(f"There are {len(initial_solution.routes)} routes")
served_customers = 0
for route in initial_solution.routes:
    customers = [cust for cust in route.customers_list if cust not in data["depots"]]
    served_customers += len(customers)
    print(route.customers_list)

print(f"Total number of served customers: {served_customers}")


In [ ]:
# Calculating the late, early, ontime and left out customers
init_solution_stats = verify_time_windows(data, initial_solution, percentage=False)
print(init_solution_stats)

In [ ]:
init_solution_stats_copy = copy.deepcopy(init_solution_stats)
del init_solution_stats_copy["sum_late"]
del init_solution_stats_copy["sum_early"]
del init_solution_stats_copy["total_served"]

In [ ]:
plt.style.use("_mpl-gallery-nogrid")
colors = plt.get_cmap("Blues")(np.linspace(0.2, 0.7, len(init_solution_stats_copy)))

print(init_solution_stats_copy.keys())

fig, ax = plt.subplots(figsize=(8, 8))
ax.pie(
    init_solution_stats_copy.values(),
    labels=["Early", "Late", "On time"],
    labeldistance=None,
    autopct="%1.1f%%",
    textprops=labels_dict,
    pctdistance=0.8,
    shadow=False,
    startangle=90,
    colors=colors,
    wedgeprops={"edgecolor": "white", "linewidth": 1},
    frame=True,
)
ax.set_title("Initial solution", fontdict=title_dict)
plt.legend(**legend_dict)
plt.axis("off")

plt.show()

In [ ]:
print(f"There are {len(solution.routes)} routes")
served_customers = 0
for route in solution.routes:
    customers = [cust for cust in route.customers_list if cust not in data['depots']]
    served_customers += len(customers)
    print(route.customers_list)


print(f"Total number of served customers: {served_customers}")
# Calculating the late, early, ontime and left out customers
solution_stats = verify_time_windows(data, solution, percentage=False)
print(solution_stats)

In [ ]:
solution_stats_copy = copy.deepcopy(solution_stats)
del solution_stats_copy["sum_late"]
del solution_stats_copy["sum_early"]
del solution_stats_copy["total_served"]

In [ ]:
plt.style.use("_mpl-gallery-nogrid")
colors = plt.get_cmap("Blues")(np.linspace(0.2, 0.7, len(solution_stats_copy)))

print(solution_stats_copy.keys())

fig, ax = plt.subplots(figsize=(8, 8))
ax.pie(
    solution_stats_copy.values(),
    labels=["Early", "Late", "On time"],
    labeldistance=None,
    autopct="%1.1f%%",
    textprops=labels_dict,
    pctdistance=0.8,
    shadow=False,
    startangle=90,
    colors=colors,
    wedgeprops={"edgecolor": "white", "linewidth": 1},
    frame=True,
)
ax.set_title("Heuristic solution", fontdict=title_dict)
plt.legend(**legend_dict)
plt.axis("off")

plt.show()

In [ ]:
print(solution)
print(initial_solution)

### Solution

In [ ]:


# Example usage
# data = read_solution_format("path_to_file.txt", print_data=True)

bks = read_solution_format("/home/pettepiero/tirocinio/dial-a-ride/data/c-mdvrptw-sol/pr01.res", print_data=False)

In [ ]:
print(bks.keys())
print(len(bks['routes']))

for route in bks["routes"]:
    print(route["customers"])

In [ ]:
def plot_bks(data: dict, solution, name= "BKS", figsize=(12, 10), save=False):

    fig, ax = plt.subplots(figsize=figsize)
    cmap = plt.get_cmap("Set2", solution["n_vehicles"])
    cmap

    for idx, route in enumerate(solution["routes"]):
        ax.plot(
            [data["node_coord"][loc][0] for loc in route["customers"]],
            [data["node_coord"][loc][1] for loc in route["customers"]],
            color=cmap(idx),
            marker=".",
            label=f"Vehicle {route['vehicle']}",
        )
        for cust in route["customers"]:
            coords = data["node_coord"][cust]
            ax.plot(coords[0], coords[1], "o", c=cmap(idx))

    kwargs = dict(zorder=3, marker="X")

    for i in range(data["dimension"], data["dimension"] + data["n_depots"]):
        depot = data["node_coord"][i]
        ax.plot(depot[0], depot[1], c="tab:red", **kwargs, label=f"Depot {i}")

    ax.scatter(*data["node_coord"][0], c="tab:red", label="Depot 0", **kwargs)

    ax.set_title(f"{name}\n Total distance: {solution["solution_cost"]}")
    ax.set_xlabel("X-coordinate")
    ax.set_ylabel("Y-coordinate")
    ax.legend(frameon=False, ncol=3)

    if save:
        plt.savefig(f"./plots/{name}.png")
        plt.close()

plot_bks(data, bks, "BKS solution",)

In [ ]:
print(f"data['node_coord][0]: {data['node_coord'][0]}")

In [ ]:
print(bks)